In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Upgrading from OpenMDAO 2.10 to OpenMDAO 3

In the OpenMDAO 3.0 release, a few changes were made to the API.  In addition, we removed all
deprecation warnings and fully deprecated the old behavior for all API changes that were made
over the lifespan of OpenMDAO 2.x.  The changes are all summarized here.

```{Note}
A significant number of the single line examples are a change in class camelcase or removing an underscore
```

## Building Component Models
### Declare a Component with distributed variables

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
class DistribComp(ExplicitComponent):

    def __init__(self, size):
        super().__init__()
        self.distributed = True

```
````

````{tab-item} OpenMDAO 3.0
```
import openmdao.api as om


class DistribComp(om.ExplicitComponent):
    """Simple Distributed Component."""

    def initialize(self):
        self.options['distributed'] = True
```
````
`````


Below each tabbed example in this notebook is the runnable code from above. However, since some things have been fully deprecated, the code from OpenMDAO 2.x is no longer operational. If that is the case, we have commented out those code cell examples.

In [ ]:
import openmdao.api as om


class DistribComp(om.ExplicitComponent):

    def __init__(self, size):
        super().__init__()
        self.distributed = True

In [ ]:
import openmdao.api as om


class DistribComp(om.ExplicitComponent):
    """Simple Distributed Component."""

    def initialize(self):
        self.options['distributed'] = True

### Declare a variable that is explicitly unitless

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
prob.model.add_subsystem('tgt', om.ExecComp('y = 3 * x', x={'units': 'unitless'}))
```
````

````{tab-item} OpenMDAO 3.0
```
prob.model.add_subsystem('tgt', om.ExecComp('y = 3 * x', x={'units': None}))
```
````
`````


In [ ]:
prob = om.Problem()
prob.model.add_subsystem('tgt', om.ExecComp('y = 3 * x', x={'units': 'unitless'}))

In [ ]:
prob = om.Problem()
prob.model.add_subsystem('tgt', om.ExecComp('y = 3 * x', x={'units': None}))

### Add a subsystem to a Group

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
indeps = prob.model.add('indeps', om.IndepVarComp())
```
````

````{tab-item} OpenMDAO 3.0
```
prob.model.add_subsystem('parab', Paraboloid(), promotes_inputs=['x', 'y'])
```
````
`````


In [ ]:
# This will result in an AttributeError
# prob = om.Problem()
# indeps = prob.model.add('indeps', om.IndepVarComp())

In [ ]:
from openmdao.test_suite.components.paraboloid import Paraboloid

prob = om.Problem()
prob.model.add_subsystem('parab', Paraboloid(), promotes_inputs=['x', 'y'])

### Add a linear or nonlinear solver to a Group

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
self.nl_solver = om.NewtonSolver()
self.ln_solver = om.DirectSolver()
```
````

````{tab-item} OpenMDAO 3.0
```
self.nonlinear_solver = om.NewtonSolver()
self.linear_solver = om.DirectSolver()
```
````
`````


In [ ]:
nl_solver = om.NewtonSolver()
ln_solver = om.DirectSolver()

In [ ]:
nonlinear_solver = om.NewtonSolver()
linear_solver = om.DirectSolver()

### Declare an option with an explicit type

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
def initialize(self):
    """
    Declare options.
    """
    self.options.declare('vec_size', type_=int, default=1,
                         desc='The number of points at which the vector magnitude is computed')
```
````

````{tab-item} OpenMDAO 3.0
```
def initialize(self):
    """
    Declare options.
    """
    self.options.declare('vec_size', types=int, default=1,
                         desc='The number of points at which the vector magnitude is computed')
```
````
`````


In [ ]:
def initialize(self):
    """
    Declare options.
    """
    self.options.declare('vec_size', type_=int, default=1,
                         desc='The number of points at which the vector magnitude is computed')

In [ ]:
def initialize(self):
    """
    Declare options.
    """
    self.options.declare('vec_size', types=int, default=1,
                         desc='The number of points at which the vector magnitude is computed')

## Component Library
### Create an interpolating component using Akima spline with uniform grid

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
import openmdao.api as om

ycp = np.array([5.0, 12.0, 14.0, 16.0, 21.0, 29.0])
ncp = len(ycp)
n = 11

prob = om.Problem()

comp = om.AkimaSplineComp(num_control_points=ncp, num_points=n,
                          name='chord')

prob.model.add_subsystem('comp1', comp)

prob.setup()
prob['akima.chord:y_cp'] = ycp.reshape((1, ncp))
prob.run_model()
```
````

````{tab-item} OpenMDAO 3.0
```
ycp = np.array([5.0, 12.0, 14.0, 16.0, 21.0, 29.0])
ncp = len(ycp)
n = 11

prob = om.Problem()

akima_option = {'delta_x': 0.1}
comp = om.SplineComp(method='akima', num_cp=ncp, x_interp_val=np.linspace(0.0, 1.0, n),
                     interp_options=akima_option)

prob.model.add_subsystem('comp1', comp)

comp.add_spline(y_cp_name='chord_cp', y_interp_name='chord', y_cp_val=ycp)

prob.setup()
prob.run_model()
```
````
`````


In [ ]:
# import numpy as np
# AkimaSplineComp is no longer a valid class


# ycp = np.array([5.0, 12.0, 14.0, 16.0, 21.0, 29.0])
# ncp = len(ycp)
# n = 11

# prob = om.Problem()

# comp = om.AkimaSplineComp(num_control_points=ncp, num_points=n,
#                           name='chord')

# prob.model.add_subsystem('comp1', comp)

# prob.setup()
# prob['akima.chord:y_cp'] = ycp.reshape((1, ncp))
# prob.run_model()

In [ ]:
import numpy as np

ycp = np.array([5.0, 12.0, 14.0, 16.0, 21.0, 29.0])
ncp = len(ycp)
n = 11

prob = om.Problem()

akima_option = {'delta_x': 0.1}
comp = om.SplineComp(method='akima', num_cp=ncp, x_interp_val=np.linspace(0.0, 1.0, n),
                     interp_options=akima_option)

prob.model.add_subsystem('comp1', comp)

comp.add_spline(y_cp_name='chord_cp', y_interp_name='chord', y_cp_val=ycp)

prob.setup()
prob.run_model()

### Create an interpolating component using Akima spline with custom grid

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
xcp = np.array([1.0, 2.0, 4.0, 6.0, 10.0, 12.0])
ycp = np.array([5.0, 12.0, 14.0, 16.0, 21.0, 29.0])
ncp = len(xcp)
n = 50
x = np.linspace(1.0, 12.0, n)

prob = om.Problem()

comp = om.AkimaSplineComp(num_control_points=ncp, num_points=n,
                          name='chord', input_x=True,
                          input_xcp=True)

prob.model.add_subsystem('akima', comp)

prob.setup(force_alloc_complex=True)

prob['akima.chord:x_cp'] = xcp
prob['akima.chord:y_cp'] = ycp.reshape((1, ncp))
prob['akima.chord:x'] = x

prob.run_model()
```
````

````{tab-item} OpenMDAO 3.0
```
xcp = np.array([1.0, 2.0, 4.0, 6.0, 10.0, 12.0])
ycp = np.array([5.0, 12.0, 14.0, 16.0, 21.0, 29.0])
n = 50
x = np.linspace(1.0, 12.0, n)

prob = om.Problem()

akima_option = {'delta_x': 0.1}
comp = om.SplineComp(method='akima', x_cp_val=xcp, x_interp_val=x,
                     interp_options=akima_option)

prob.model.add_subsystem('akima1', comp)

comp.add_spline(y_cp_name='ycp', y_interp_name='y_val', y_cp_val=ycp)

prob.setup(force_alloc_complex=True)
prob.run_model()
```
````
`````


In [ ]:
# AkimaSplineComp is no longer a valid class


# xcp = np.array([1.0, 2.0, 4.0, 6.0, 10.0, 12.0])
# ycp = np.array([5.0, 12.0, 14.0, 16.0, 21.0, 29.0])
# ncp = len(xcp)
# n = 50
# x = np.linspace(1.0, 12.0, n)

# prob = om.Problem()

# comp = om.AkimaSplineComp(num_control_points=ncp, num_points=n,
#                           name='chord', input_x=True,
#                           input_xcp=True)

# prob.model.add_subsystem('akima', comp)

# prob.setup(force_alloc_complex=True)

# prob['akima.chord:x_cp'] = xcp
# prob['akima.chord:y_cp'] = ycp.reshape((1, ncp))
# prob['akima.chord:x'] = x

# prob.run_model()

In [ ]:
xcp = np.array([1.0, 2.0, 4.0, 6.0, 10.0, 12.0])
ycp = np.array([5.0, 12.0, 14.0, 16.0, 21.0, 29.0])
n = 50
x = np.linspace(1.0, 12.0, n)

prob = om.Problem()

akima_option = {'delta_x': 0.1}
comp = om.SplineComp(method='akima', x_cp_val=xcp, x_interp_val=x,
                     interp_options=akima_option)

prob.model.add_subsystem('akima1', comp)

comp.add_spline(y_cp_name='ycp', y_interp_name='y_val', y_cp_val=ycp)

prob.setup(force_alloc_complex=True)
prob.run_model()

### Create an interpolating component using Bsplines

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
prob = om.Problem()
model = prob.model

n_cp = 5
n_point = 10

t = np.linspace(0, 0.5*np.pi, n_cp)
x = np.empty((2, n_cp))
x[0, :] = np.sin(t)
x[1, :] = 2.0*np.sin(t)

comp = om.BsplinesComp(num_control_points=n_cp,
                       num_points=n_point,
                       bspline_order=4,
                       distribution='sine',
                       vec_size=2,
                       in_name='h_cp',
                       out_name='h')

model.add_subsystem('interp', comp)

prob.setup()
prob.run_model()
```
````

````{tab-item} OpenMDAO 3.0
```
from openmdao.utils.spline_distributions import sine_distribution

prob = om.Problem()
model = prob.model

n_cp = 5
n_point = 10

t = np.linspace(0, 0.5 * np.pi, n_cp)
x = np.empty((2, n_cp))
x[0, :] = np.sin(t)
x[1, :] = 2.0 * np.sin(t)

# In 2.x, the BsplinesComp had a built-in sinusoidal distribution.
t_sin = sine_distribution(n_point) * np.pi * 0.5

bspline_options = {'order': 4}
comp = om.SplineComp(method='bsplines',
                     x_interp_val=t_sin,
                     num_cp=n_cp,
                     vec_size=2,
                     interp_options=bspline_options)

prob.model.add_subsystem('interp', comp)

comp.add_spline(y_cp_name='h_cp', y_interp_name='h', y_cp_val=x, y_units='km')

prob.setup()
prob.run_model()
```
````
`````


In [ ]:
# BSplineComp is no longer a valid class

# prob = om.Problem()
# model = prob.model

# n_cp = 5
# n_point = 10

# t = np.linspace(0, 0.5*np.pi, n_cp)
# x = np.empty((2, n_cp))
# x[0, :] = np.sin(t)
# x[1, :] = 2.0*np.sin(t)

# comp = om.BsplinesComp(num_control_points=n_cp,
#                        num_points=n_point,
#                        bspline_order=4,
#                        distribution='sine',
#                        vec_size=2,
#                        in_name='h_cp',
#                        out_name='h')

# model.add_subsystem('interp', comp)

# prob.setup()
# prob.run_model()

In [ ]:
from openmdao.utils.spline_distributions import sine_distribution

prob = om.Problem()
model = prob.model

n_cp = 5
n_point = 10

t = np.linspace(0, 0.5 * np.pi, n_cp)
x = np.empty((2, n_cp))
x[0, :] = np.sin(t)
x[1, :] = 2.0 * np.sin(t)

# In 2.x, the BsplinesComp had a built-in sinusoidal distribution.
t_sin = sine_distribution(n_point) * np.pi * 0.5

bspline_options = {'order': 4}
comp = om.SplineComp(method='bsplines',
                     x_interp_val=t_sin,
                     num_cp=n_cp,
                     vec_size=2,
                     interp_options=bspline_options)

prob.model.add_subsystem('interp', comp)

comp.add_spline(y_cp_name='h_cp', y_interp_name='h', y_cp_val=x, y_units='km')

prob.setup()
prob.run_model()

### Create an ExecComp with diagonal partials

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
model.add_subsystem('comp', ExecComp('y=3.0*x + 2.5',
                                     vectorize=True,
                                     x=np.ones(5), y=np.ones(5)))
```
````

````{tab-item} OpenMDAO 3.0
```
model.add_subsystem('comp', om.ExecComp('y=3.0*x + 2.5',
                                        has_diag_partials=True,
                                        x=np.ones(5), y=np.ones(5)))
```
````
`````


In [ ]:
p = om.Problem()
model = p.model

model.add_subsystem('comp', om.ExecComp('y=3.0*x + 2.5',
                                     vectorize=True,
                                     x=np.ones(5), y=np.ones(5)))

In [ ]:
p = om.Problem()
model = p.model

model.add_subsystem('comp', om.ExecComp('y=3.0*x + 2.5',
                                        has_diag_partials=True,
                                        x=np.ones(5), y=np.ones(5)))

### Create an IndepVarComp with multiple outputs

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
comp = om.IndepVarComp((
    ('indep_var_1', 1.0, {'lower': 0, 'upper': 10}),
    ('indep_var_2', 2.0, {'lower': 1., 'upper': 20}),
))
```
````

````{tab-item} OpenMDAO 3.0
```
comp = om.IndepVarComp()
comp.add_output('indep_var_1', val=1.0)
comp.add_output('indep_var_2', val=2.0)
```
````
`````


In [ ]:
# comp = om.IndepVarComp((
#     ('indep_var_1', 1.0, {'lower': 0, 'upper': 10}),
#     ('indep_var_2', 2.0, {'lower': 1., 'upper': 20}),
# ))

In [ ]:
comp = om.IndepVarComp()
comp.add_output('indep_var_1', val=1.0)
comp.add_output('indep_var_2', val=2.0)

### Create an ExternalCodeComp

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
class ParaboloidExternalCodeCompDerivs(om.ExternalCode):
```
````

````{tab-item} OpenMDAO 3.0
```
class ParaboloidExternalCodeCompDerivs(om.ExternalCodeComp):
```
````
`````


In [ ]:
# class ParaboloidExternalCodeCompDerivs(om.ExternalCode):
#     pass

In [ ]:
class ParaboloidExternalCodeCompDerivs(om.ExternalCodeComp):
    pass

### Create a KSComponent

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
model.add_subsystem('ks', om.KSComponent(width=2))
```
````

````{tab-item} OpenMDAO 3.0
```
model.add_subsystem('ks', om.KSComp(width=2))
```
````
`````


In [ ]:
# model = om.Problem().model
# model.add_subsystem('ks', om.KSComponent(width=2))

In [ ]:
model = om.Problem().model
model.add_subsystem('ks', om.KSComp(width=2))

### Create a MetaModel

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
sin_mm = om.MetaModel()
```
````

````{tab-item} OpenMDAO 3.0
```
sin_mm = om.MetaModelUnStructuredComp()
```
````
`````


In [ ]:
# sin_mm = om.MetaModel()

In [ ]:
sin_mm = om.MetaModelUnStructuredComp()

### Create a MetaModelUnstructured

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
sin_mm = om.MetaModelUnstructured()
```
````

````{tab-item} OpenMDAO 3.0
```
sin_mm = om.MetaModelUnStructuredComp()
```
````
`````


In [ ]:
# sin_mm = om.MetaModelUnstructured()

In [ ]:
sin_mm = om.MetaModelUnStructuredComp()

### Create a MetaModelStructured

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
interp = om.MetaModelStructured(method='scipy_cubic', vec_size=2)
```
````

````{tab-item} OpenMDAO 3.0
```
interp = om.MetaModelStructuredComp(method='scipy_cubic', vec_size=2)
```
````
`````


In [ ]:
# interp = om.MetaModelStructured(method='scipy_cubic', vec_size=2)

In [ ]:
interp = om.MetaModelStructuredComp(method='scipy_cubic', vec_size=2)

### Create a MultiFiMetaModel

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
mm = om.MultiFiMetaModel(nfi=2)
```
````

````{tab-item} OpenMDAO 3.0
```
mm = om.MultiFiMetaModelUnStructuredComp(nfi=2)
```
````
`````


In [ ]:
# mm = om.MultiFiMetaModel(nfi=2)

In [ ]:
mm = om.MultiFiMetaModelUnStructuredComp(nfi=2)

### Create a MultiFiMetaModelUnStructured

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
mm = om.MultiFiMetaModelUnStructured(nfi=2)
```
````

````{tab-item} OpenMDAO 3.0
```
mm = om.MultiFiMetaModelUnStructuredComp(nfi=2)
```
````
`````


In [ ]:
# mm = om.MultiFiMetaModelUnStructured(nfi=2)

In [ ]:
mm = om.MultiFiMetaModelUnStructuredComp(nfi=2)

### Add a FloatKrigingSurrogate to a MetaModelStructuredComp

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
sin_mm.add_output('f_x', 0., surrogate=om.FloatKrigingSurrogate())
```
````

````{tab-item} OpenMDAO 3.0
```
sin_mm.add_output('f_x', 0., surrogate=om.KrigingSurrogate())
```
````
`````


In [ ]:
# sin_mm.add_output('f_x', 0., surrogate=om.FloatKrigingSurrogate())

In [ ]:
sin_mm.add_output('f_x', 0., surrogate=om.KrigingSurrogate())

### Specify a default surrogate model for MetaModelStructuredComp

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
trig = om.MetaModelUnStructuredComp(vec_size=size)
trig.default_surrogate = om.KrigingSurrogate()
```
````

````{tab-item} OpenMDAO 3.0
```
trig = om.MetaModelUnStructuredComp(vec_size=size, default_surrogate=om.KrigingSurrogate())
```
````
`````


In [ ]:
trig = om.MetaModelUnStructuredComp(vec_size=2)
trig.default_surrogate = om.KrigingSurrogate()

In [ ]:
trig = om.MetaModelUnStructuredComp(vec_size=2, default_surrogate=om.KrigingSurrogate())

## Solvers
### Declare a NewtonSolver with solve_subsystems set to False

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
newton = model.nonlinear_solver = om.NewtonSolver()
```
````

````{tab-item} OpenMDAO 3.0
```
newton = model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
```
````
`````


In [ ]:
newton = model.nonlinear_solver = om.NewtonSolver()

In [ ]:
newton = model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)

### Control how a solver handles an error raised in a subsolver

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
newton = model.nonlinear_solver = NewtonSolver()
newton.options['maxiter'] = 1
newton.options['err_on_maxiter'] = True
```
````

````{tab-item} OpenMDAO 3.0
```
newton = model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
newton.options['maxiter'] = 1
newton.options['err_on_non_converge'] = True
```
````
`````


In [ ]:
# newton = model.nonlinear_solver = om.NewtonSolver()
# newton.options['maxiter'] = 1
# newton.options['err_on_maxiter'] = True

In [ ]:
newton = model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
newton.options['maxiter'] = 1
newton.options['err_on_non_converge'] = True

### Declare a BroydenSolver with the BoundsEnforceLS line search

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
model.circuit.nonlinear_solver = om.BroydenSolver()
model.circuit.nonlinear_solver.linesearch = om.BoundsEnforceLS()
```
````

````{tab-item} OpenMDAO 3.0
```
model.nonlinear_solver = om.BroydenSolver()
```
````
`````


In [ ]:
model = om.Problem().model
model.nonlinear_solver = om.BroydenSolver()
model.nonlinear_solver.linesearch = om.BoundsEnforceLS()

In [ ]:
model = om.Problem().model
model.nonlinear_solver = om.BroydenSolver()

### Declare a NewtonSolver with the BoundsEnforceLS line search

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
newton = model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
newton.linesearch = om.BoundsEnforceLS()
```
````

````{tab-item} OpenMDAO 3.0
```
model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
```
````
`````


In [ ]:
model = om.Problem().model
newton = model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
newton.linesearch = om.BoundsEnforceLS()

In [ ]:
model = om.Problem().model
model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)

### Add a preconditioner to PETScKrylov

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
model.linear_solver = om.PETScKrylov()
model.linear_solver.preconditioner = om.LinearBlockGS()
```
````

````{tab-item} OpenMDAO 3.0
```
model.linear_solver = om.PETScKrylov()
model.linear_solver.precon = om.LinearBlockGS()
```
````
`````


In [ ]:
model = om.Problem().model

# This try/except block is for our CI testing
try:
    model.linear_solver = om.PETScKrylov()
    model.linear_solver.preconditioner = om.LinearBlockGS()
except RuntimeError:
    pass

In [ ]:
model = om.Problem().model

try:
    model.linear_solver = om.PETScKrylov()
    model.linear_solver.precon = om.LinearBlockGS()
except RuntimeError:
    pass

### Add a preconditioner to ScipyKrylov

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
model.linear_solver.preconditioner = om.LinearBlockGS()
```
````

````{tab-item} OpenMDAO 3.0
```
model.linear_solver.precon = om.LinearBlockGS()
```
````
`````


In [ ]:
model = om.Problem().model

model.linear_solver.preconditioner = om.LinearBlockGS()

In [ ]:
model = om.Problem().model

model.linear_solver.precon = om.LinearBlockGS()

### Add a ArmijoGoldsteinLS to a NewtonSolver

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
top.model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
top.model.nonlinear_solver.options['maxiter'] = 10
top.model.linear_solver = om.ScipyKrylov()

ls = top.model.nonlinear_solver.line_search = om.ArmijoGoldsteinLS(bound_enforcement='vector')
```
````

````{tab-item} OpenMDAO 3.0
```
top.model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
top.model.nonlinear_solver.options['maxiter'] = 10
top.model.linear_solver = om.ScipyKrylov()

ls = top.model.nonlinear_solver.linesearch = om.ArmijoGoldsteinLS(bound_enforcement='vector')
```
````
`````


In [ ]:
top = om.Problem()

top.model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
top.model.nonlinear_solver.options['maxiter'] = 10
top.model.linear_solver = om.ScipyKrylov()

ls = top.model.nonlinear_solver.line_search = om.ArmijoGoldsteinLS(bound_enforcement='vector')

In [ ]:
top = om.Problem()

top.model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
top.model.nonlinear_solver.options['maxiter'] = 10
top.model.linear_solver = om.ScipyKrylov()

ls = top.model.nonlinear_solver.linesearch = om.ArmijoGoldsteinLS(bound_enforcement='vector')

### Create a NonLinearRunOnce

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
model.nonlinear_solver = om.NonLinearRunOnce()
```
````

````{tab-item} OpenMDAO 3.0
```
model.nonlinear_solver = om.NonlinearRunOnce()
```
````
`````


In [ ]:
# model = om.Problem().model
# model.nonlinear_solver = om.NonLinearRunOnce()

In [ ]:
model = om.Problem().model
model.nonlinear_solver = om.NonlinearRunOnce()

### Create a PetscKSP

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
model.linear_solver = om.PetscKSP()
```
````

````{tab-item} OpenMDAO 3.0
```
model.linear_solver = om.PETScKrylov()
```
````
`````


In [ ]:
# model = om.Problem().model

# model.linear_solver = om.PetscKSP()

In [ ]:
model = om.Problem().model
try:
    model.linear_solver = om.PETScKrylov()
except RuntimeError:
    pass

### Create a ScipyIterativeSolver

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
model.linear_solver = om.ScipyIterativeSolver()
```
````

````{tab-item} OpenMDAO 3.0
```
model.linear_solver = om.ScipyKrylov()
```
````
`````


In [ ]:
# model = om.Problem().model
# model.linear_solver = om.ScipyIterativeSolver()

In [ ]:
model = om.Problem().model
model.linear_solver = om.ScipyKrylov()

## Drivers
### Activate dynamic coloring on a Driver

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
p.driver.options['dynamic_simul_derivs'] = True
```
````

````{tab-item} OpenMDAO 3.0
```
p.driver.declare_coloring()
```
````
`````


In [ ]:
# p = om.Problem()
# p.driver.options['dynamic_simul_derivs'] = True

In [ ]:
p = om.Problem()
p.driver.declare_coloring()

### Add a ScipyOptimizer to a Problem

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
prob.driver = om.ScipyOptimizer()
```
````

````{tab-item} OpenMDAO 3.0
```
prob.driver = om.ScipyOptimizeDriver()
```
````
`````


In [ ]:
# prob = om.Problem()
# prob.driver = om.ScipyOptimizer()

In [ ]:
prob = om.Problem()
prob.driver = om.ScipyOptimizeDriver()

## Working with Derivatives
### Use a pre-computed coloring on a model

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
p.driver.set_simul_deriv_color()
```
````

````{tab-item} OpenMDAO 3.0
```
p.driver.use_fixed_coloring()
```
````
`````


In [ ]:
# p = om.Problem()
# p.driver = om.ScipyOptimizeDriver()
# p.driver.set_simul_deriv_color()

In [ ]:
p = om.Problem()
p.driver = om.ScipyOptimizeDriver()
p.driver.use_fixed_coloring()

## Case Reading
### Query the iteration coordinate for a case

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
cr = om.CaseReader(self.filename)

for i, c in enumerate(cr.list_cases()):
    case = cr.get_case(c)

    coord = case.iteration_coordinate
```
````

````{tab-item} OpenMDAO 3.0
```
cr = om.CaseReader(filename)

for i, c in enumerate(cr.list_cases(out_stream=None)):
    case = cr.get_case(c)

    coord = case.name
```
````
`````


In [ ]:
# This is to create a CaseRecorder
from openmdao.test_suite.components.implicit_newton_linesearch import ImplCompTwoStates

filename = "sqlite_test"
recorder = om.SqliteRecorder(filename, record_viewer_data=False)

prob = om.Problem()

model = prob.model

model.add_subsystem('px', om.IndepVarComp('x', 1.0))
model.add_subsystem('comp', ImplCompTwoStates())
model.connect('px.x', 'comp.x')

model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
model.nonlinear_solver.options['maxiter'] = 3
model.nonlinear_solver.options['iprint'] = 2
model.linear_solver = om.ScipyKrylov()

ls = model.nonlinear_solver.linesearch = om.ArmijoGoldsteinLS(bound_enforcement='vector')
ls.options['maxiter'] = 3
ls.options['alpha'] = 1.0

# add recorder to nonlinear solver, linesearch solver and model
model.nonlinear_solver.add_recorder(recorder)
model.nonlinear_solver.linesearch.add_recorder(recorder)
model.comp.add_recorder(recorder)
model.add_recorder(recorder)

prob.setup()
prob.set_solver_print(0)

prob['px.x'] = 2.0
prob['comp.y'] = 0.0
prob['comp.z'] = 1.6
prob.run_model()
prob.cleanup()

In [ ]:
# cr = om.CaseReader(filename)

# for i, c in enumerate(cr.list_cases()):
#     case = cr.get_case(c)

#     coord = case.iteration_coordinate

In [ ]:
cr = om.CaseReader(filename)

for i, c in enumerate(cr.list_cases(out_stream=None)):
    case = cr.get_case(c)

    coord = case.name

## Running a Model
### Run a Driver

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
prob.run()
```
````

````{tab-item} OpenMDAO 3.0
```
prob.run_driver()
```
````
`````


In [ ]:
# prob = om.Problem()
# prob.run()

In [ ]:
prob = om.Problem()
prob.setup()
prob.run_driver();

### Run a Model without Running the Driver

`````{tab-set}
````{tab-item} OpenMDAO 2.x
```
prob.run_once()
```
````

````{tab-item} OpenMDAO 3.0
```
prob.run_model()
```
````
`````


In [ ]:
# prob = om.Problem()
# prob.run_once()

In [ ]:
prob = om.Problem()
prob.setup()
prob.run_model()